### WebSocket Server

Documentation WebsocketServer, see https://github.com/Pithikos/python-websocket-server

In [21]:
import logging
from websocket_server import WebsocketServer

In [22]:
def new_client(client, server):
    player_enters(client, server)
    server.send_message_to_all("Hey all, a new client has joined us")

def client_left(client, server):
    player_leaves(client, server)

def message_received(client, server, message):
    # Split message, and omit empty messages
    s = message.split()
    if not s:
        return
    
    # Find command
    c, args = s[0], s[1:]
    if c == 'create':
        return player_create_room(client, server, args)    
    if c == 'join':
        return player_join_room(client, server, args)
    if c == 'name':
        return player_name(client, server, args)
    if c == 'move':
        return player_move_cards(client, server, args)
    if c == 'face':
        return player_face_cards(client, server, args)
    if c == 'place':
        return player_place_cards(client, server, args)
    if c == 'shuffle':
        return player_shuffle_cards(client, server, args)
    
    # Invalid command
    server.send_message(client, 'error invalid command {}'.format(c))

In [ ]:
# Create House
house = House()

# Start server
server = WebsocketServer(45311, host = '0.0.0.0', loglevel = logging.INFO)
server.set_fn_new_client(new_client)
server.set_fn_client_left(client_left)
server.set_fn_message_received(message_received)
server.run_forever()

INFO:websocket_server.websocket_server:Listening on port 45311 for clients..
INFO:websocket_server.websocket_server:Client asked to close connection.
INFO:websocket_server.websocket_server:Client asked to close connection.
INFO:websocket_server.websocket_server:Client asked to close connection.
INFO:websocket_server.websocket_server:Client asked to close connection.
INFO:websocket_server.websocket_server:Client asked to close connection.


### Game Engine

In [3]:
# Imports
import random
import string

# Constants
SUITS = ['S', 'C', 'H', 'D']
NUMBERS = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
FACE_UP = 'U'
FACE_DOWN = 'D'
TABLE_ID = -1

# Util functions
def create_random_id():
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k = 6))

def str_is_float(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [4]:
# Commands
def player_create_room(client, server, args):
    if len(args) != 0:
        server.send_message(client, 'error create requires 0 arguments')
        return

    room_id = house.create_room()
    player_join_room(client, server, [ room_id ])
    
def player_join_room(client, server, args):
    if len(args) != 1:
        server.send_message(client, 'error join requires 1 argument')
        return
    
    room_id = args[0]
    if room_id not in house.rooms:
        server.send_message(client, 'error room {} does not exist'.format(room_id))
        return

    player_id = client['id']
    if player_id in house.player_to_room:
        house.player_to_room[player_id].remove_player(server, player_id)
    server.send_message(client, 'room {}'.format(room_id))
    house.rooms[room_id].add_player(server, player_id)
    house.player_to_room[player_id] = house.rooms[room_id]
    
def player_name(client, server, args):
    if len(args) != 1:
        server.send_message(client, 'error name requires 1 argument')
        return
        
    player_id = client['id']
    house.player_to_name[player_id] = args[0]
    server.send_message(client, 'ok')
    
    if player_id in house.player_to_room:
        house.player_to_room[player_id].update_name(server, player_id)
    
def player_move_cards(client, server, args):
    if len(args) < 3:
        server.send_message(client, 'error move requires at least 3 argument')
        return

    if client['id'] not in house.player_to_room:
        server.send_message(client, 'error move requires to be in a room')
        return
    
    room = house.player_to_room[client['id']]
    
    if not str_is_float(args[0]) or not str_is_float(args[1]):
        server.send_message(client, 'error argument 1 and 2 of move should be floats')
        return
    
    x, y = float(args[0]), float(args[1])
    
    ids = [ int(s) for s in args[2:] if s.isnumeric() ] # TODO: just omit non-numeric arguments?
    ids = [ i for i in ids if i in room.cards ]
    for i in ids:
        card = room.cards[i]
        if card.place == TABLE_ID or card.place == client['id']:
            card.position = (x, y)
            
    room.update_cards(server, ids)
    
def player_face_cards(client, server, args):
    if len(args) < 2:
        server.send_message(client, 'error face requires at least 2 argument')
        return

    if client['id'] not in house.player_to_room:
        server.send_message(client, 'error face requires to be in a room')
        return
    
    room = house.player_to_room[client['id']]
    
    face = args[0]
    if face not in [ FACE_UP, FACE_DOWN ]:
        server.send_message(client, 'error first argument of face should be {} or {}'.format(FACE_UP, FACE_DOWN))
        return
    
    ids = [ int(s) for s in args[1:] if s.isnumeric() ] # TODO: just omit non-numeric arguments?
    ids = [ i for i in ids if i in room.cards ]
    for i in ids:
        card = room.cards[i]
        if card.place == TABLE_ID or card.place == client['id']:
            card.face = face
            
    room.update_cards(server, ids)
            
def player_place_cards(client, server, args):
    if len(args) < 2:
        server.send_message(client, 'error place requires at least 2 arguments')
        return

    if client['id'] not in house.player_to_room:
        server.send_message(client, 'error place requires to be in a room')
        return
    
    room = house.player_to_room[client['id']]
    
    if not args[0].isnumeric() and args[0] != '-1':
        server.send_message(client, 'error first argument place should be numeric')
        return
    
    place = int(args[0])
    if place not in room.players and place != TABLE_ID :
        server.send_message(client, 'error player {} is not in this room'.format(place))
        return
        
    ids = [ int(s) for s in args[1:] if s.isnumeric() ] # TODO: just omit non-numeric arguments?
    ids = [ i for i in ids if i in room.cards ]
    for i in ids:
        card = room.cards[i]
        if card.place == TABLE_ID or card.place == client['id']:
            room.cards[i].place = place
            
    room.update_cards(server, ids)
    
def player_shuffle_cards(client, server, args):
    if len(args) < 1:
        server.send_message(client, 'error shuffle requires at least 1 argument')
        return

    if client['id'] not in house.player_to_room:
        server.send_message(client, 'error shuffle requires to be in a room')
        return
    
    room = house.player_to_room[client['id']]
        
    ids = [ int(s) for s in args[1:] if s.isnumeric() ] # TODO: just omit non-numeric arguments?
    ids = [ i for i in ids if i in room.cards and room.cards[i].place == TABLE_ID ] # Only shuffle cards that are on the table
    cards = [ room.cards[i] for i in ids ]
    
    if len(cards) == 0:
        return
    
    random.shuffle(cards)
    first_value = cards[0].value
    for i in range(len(cards) - 1):
        cards[i].value = cards[i + 1].value
    cards[-1].value = first_value
    
    room.update_cards(server, ids)

In [5]:
# Other functions
def player_enters(client, server):
    player_id = client['id']
    house.player_to_client[player_id] = client
    house.player_to_name[player_id] = 'Player ' + str(player_id)
    server.send_message(client, 'welcome {}'.format(player_id))

def player_leaves(client, server):
    player_id = client['id']
    if player_id in house.player_to_room:
        house.player_to_room[player_id].remove_player(server, player_id)
        del house.player_to_room[player_id]

In [6]:
class House:
    
    def __init__(self):
        self.rooms = {} # Dictionary { room id: Room }
        self.player_to_client = {} # Dictionary { player id: client }
        self.player_to_name = {} # Dictionary { player id: name }
        self.player_to_room = {} # Dictionary { player id: room }

    def create_room(self):
        room_id = create_random_id()
        room = Room()
        room.reset()
        self.rooms[room_id] = room
        return room_id

In [7]:
class Card:

    def __init__(self, value):
        self.value = value
        self.place = TABLE_ID
        self.position = (0.0, 0.0)
        self.face = FACE_DOWN

In [20]:
class Room:
    
    def __init__(self):
        self.players = set() # Set of player ids in the room
        self.cards = {} # Dictionary { card id: Card }
        
    def reset(self):
        # Place deck of cards in center of table
        self.cards = {}
        i = 0
        for suit in SUITS:
            for number in NUMBERS:
                self.cards[i] = Card(suit + number)
                i += 1
    
    def add_player(self, server, player_id):
        self.players.add(player_id)
        
        # Send players / names messages
        players_message = 'players {}'.format(' '.join([ str(i) for i in self.players ]))
        for p in self.players:
            server.send_message(house.player_to_client[p], players_message)
            server.send_message(house.player_to_client[p], 'name {} {}'.format(player_id, house.player_to_name[player_id]))
            server.send_message(house.player_to_client[player_id], 'name {} {}'.format(p, house.player_to_name[p]))
        
        # Send card data to new player
        server.send_message(house.player_to_client[player_id], 'cards {}'.format(len(self.cards)))
        for i in self.cards:
            card = self.cards[i]
            card_value = card.value if card.face == FACE_UP else '?'
            server.send_message(house.player_to_client[player_id], 'card {} {} {} {} {} {}'.format(i, card_value, card.place, card.position[0], card.position[1], card.face))
    
    def remove_player(self, server, player_id):
        if player_id in self.players:
            self.players.remove(player_id)
                        
        # Cards in player's hand go onto table
        for i in self.cards:
            if self.cards[i].place == player_id:
                self.cards[i].place = TABLE_ID
                
        message = 'players {}'.format(' '.join([ str(i) for i in self.players ]))
        for p in self.players:
            server.send_message(house.player_to_client[p], message)
            
    def update_cards(self, server, ids):
        for i in [ i for i in ids if i in self.cards ]:
            card = self.cards[i]
            card_value = card.value if card.face == FACE_UP else '?'
            message = 'card {} {} {} {} {} {}'.format(i, card_value, card.place, card.position[0], card.position[1], card.face)
            for p in self.players:
                server.send_message(house.player_to_client[p], message)
                
    def update_name(self, server, player_id):
        if player_id not in self.players:
            return
        
        for p in self.players:
            server.send_message(house.player_to_client[p], 'name {} {}'.format(player_id, house.player_to_name[player_id]))
